# File paths and environment variables

In [81]:
from pathlib import Path
import pandas as pd
import numpy as np

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

""" ============================================ EDIT THESE SETTINGS BASED ON USER'S CHOICE. ============================================ """
""" RefSeq catalog settings"""
TARGET_DIR = Path("/mnt/e/infant_nt/database")  # the base directory for everything else.
TARGET_TAXA = "Enterococcaceae"  # the taxonomic identifier to download. Can be species, genus or even family.
NCBI_REFSEQ_DIR = TARGET_DIR / "ref_genomes"
REFSEQ_INDEX = NCBI_REFSEQ_DIR / "index.tsv"

""" RefSeq BLAST database """
BLAST_DB_DIR = TARGET_DIR / "blast_db"
BLAST_DB_NAME = "Efaecalis_refseq"  # Blast DB to create.

""" Marker seeds """
MARKER_SEED_DIR = TARGET_DIR / "marker_seeds"
MARKER_SEED_INDEX = MARKER_SEED_DIR / "marker_seed_index.tsv"

""" chronostrain-specific settings """
NUM_CORES = 8  # number of cores to use (e.g. for blastn)
MIN_PCT_IDTY = 75  # accept BLAST hits as markers above this threshold.
CHRONOSTRAIN_DB_DIR = TARGET_DIR / "chronostrain_files"  # The directory to use for chronostrain's database files.
CHRONOSTRAIN_TARGET_JSON = CHRONOSTRAIN_DB_DIR / "efaecalis.json"  # the desired final product.
DASHING2_DIR = Path("/home/youn/work/bin")  # Directory that contains the dashing2 executable.

""" MetaPhlAn settings """
METAPHLAN_DB_PATH = Path("/mnt/e/metaphlan_databases/mpa_vJan21_CHOCOPhlAnSGB_202103/mpa_vJan21_CHOCOPhlAnSGB_202103.pkl") # MetaPhlan 3 or newer
METAPHLAN_TAXONOMIC_KEY = 's__Enterococcus_faecalis'

""" ============================================ DO NOT EDIT BELOW ============================================ """
""" environment variable extraction """
try:
    VARS_SET
except NameError:
    VARS_SET = True
    _cwd = %pwd
    _parent_cwd = Path(_cwd).parent
    _start_path = %env PATH

# Work in parent directory, where all the helper scripts and settings.sh are.
%cd "$_parent_cwd"
# Don't use GPU when importing jaxlib through chronostrain.
%env JAX_PLATFORM_NAME=cpu  
%env TARGET_TAXA=$TARGET_TAXA
%env NCBI_REFSEQ_DIR=$NCBI_REFSEQ_DIR
%env REFSEQ_INDEX=$REFSEQ_INDEX
# Need basic executables, such as "which" and "basename" (required by primersearch)
%env PATH=/usr/bin:$_start_path:$DASHING2_DIR

/home/youn/work/chronostrain/examples/database
env: JAX_PLATFORM_NAME=cpu
env: TARGET_TAXA=Enterococcaceae
env: NCBI_REFSEQ_DIR=/mnt/e/infant_nt/database/ref_genomes
env: REFSEQ_INDEX=/mnt/e/infant_nt/database/ref_genomes/index.tsv
env: PATH=/usr/bin:/home/youn/mambaforge/envs/chronostrain/bin:/home/youn/work/bin


In [86]:
# === Ensure that these commands work.
!primersearch --version
!dashing2 --version

EMBOSS:6.6.0.0
#Calling Dashing2 version v2.1.19 with command '/home/youn/work/chronostrain/examples/database/dashing2 --version'
dashing2 has several subcommands: sketch, cmp, wsketch, and contain.
Usage can be seen in those subcommands. (e.g., `dashing2 sketch -h`)

	sketch: converts FastX into k-mer sets/sketches, and sketches BigWig and BED files; also contains functionality from cmp, for one-step sketch and comparisons
This is probably the most common subcommand to use.

	cmp: compares previously sketched/decomposed k-mer sets and emits results. alias: dist

	contain: Takes a k-mer database (built with dashing2 sketch --save-kmers), then computes coverage for all k-mer references using input streams.
	wsketch: Takes a tuple of [1-3] input binary files [(u32 or u64), (float or double), (u32 or u64)] and performs weighted minhash sketching.
Three files are treated as Compressed Sparse Row (CSR)-format, where the third file contains indptr values, specifying the lengths of consecutiv

# Recipe starts here.

In [2]:
# Prepare directories.
TARGET_DIR.mkdir(exist_ok=True, parents=True)
BLAST_DB_DIR.mkdir(exist_ok=True, parents=True)
NCBI_REFSEQ_DIR.mkdir(exist_ok=True, parents=True)
MARKER_SEED_DIR.mkdir(exist_ok=True, parents=True)

### Step 1: Download RefSeq catalog.

In [5]:
!bash download_ncbi2.sh

2023-09-20 16:43:14,747 [INFO - chronostrain.download_ncbi] - Found 2197 records.
^C

Aborted!


### Step 2: Build Blast database.

In [ ]:
!bash create_blast_db.sh

### Step 3: Build the marker seed catalog.

ChronoStrain only needs a FASTA file of marker seeds (one multi-fasta file per marker gene), and a single TSV file that catalogs them.
However, to get there, we need to take a few steps...

#### Step 3.1: Download MLST schema.

In [13]:
!python python_helpers/mlst_download.py -t "Enterococcus faecalis" -w "$TARGET_DIR"/mlst_schema -o "$MARKER_SEED_DIR"/mlst_seeds.tsv

Targeting 1 taxa using MLST scheme.
Fetching URL resource https://pubmlst.org/static/data/dbases.xml
Got a response of size 152.35 KB.
Schema type id: 
Handling locus gdh
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/loci/gdh/alleles_fasta
Got a response of size 69.5 KB.
Handling locus gyd
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/loci/gyd/alleles_fasta
Got a response of size 20.51 KB.
Handling locus pstS
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/loci/pstS/alleles_fasta
Got a response of size 74.72 KB.
Handling locus gki
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/loci/gki/alleles_fasta
Got a response of size 54.14 KB.
Handling locus aroE
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/loci/aroE/alleles_fasta
Got a response of size 60.48 KB.
Handling locus xpt
Fetching URL resource https://rest.pubmlst.org/db/pubmlst_efaecalis_seqdef/l

#### Step 3.2: Non-standard genes: run PCR primer search across entire catalog.

In [11]:
def perform_primer_search(gene_name: str, forward_primer: str, rev_primer: str, est_length: int):
    !python python_helpers/identify_gene_cluster_primersearch.py \
        -i "$REFSEQ_INDEX" \
        -t "$TARGET_DIR"/_tmp \
        -o "$MARKER_SEED_DIR"/"$gene_name".feather \
        -g "Enterococcus" \
        -s "faecalis" \
        -p1 "$forward_primer" -p2 "$rev_primer" \
        -n "$gene_name" \
        -l "$est_length" \
        --dont-use-gff


# ====== Known strain polymorphisms
perform_primer_search("cpsA", "GTAGAAGAAGCAAGCCAGTACGCC", "CCTCTGCAGCAATCTGTTTCATGG", 478)
perform_primer_search("cpsB", "GTGTCATCACAGCTATCGTCGC", "CCGGCATTGGATAAGGAAATAGCC", 603)
perform_primer_search("cpsC", "CCTGAATATCAATGTATTTGGGCAGTC", "CCAACGCTTTGCTTCTTGAATGAC", 300)
perform_primer_search("cpsD", "GGATTCTCTTGTTCAACAAACCATTGG", "CGCATGGCTTCATAAAAGAACAGC", 522)
perform_primer_search("cpsE", "GAGGTTGAGCGAGATATATTATGGC", "CACTTCATAAACCGACTCATCACG", 450)
perform_primer_search("cpsF", "GCATTACAAGGTTATACAGTTGATGG", "GACTGTTCCATCTTATCTTTTATTCGG", 580)
perform_primer_search("cpsG", "GGCTCTGATCAAATGTGGAATCCC", "GGTGTATCTTCAGAAACATATTCTACTG", 503)
perform_primer_search("cpsH", "GTGTCTTTAGCAATTGGTATCGGTTG", "CACTAGAGTAGCTAATACTTTTTTTTCCC", 366)
perform_primer_search("cpsI", "GCTTGTGAAGCAGCTAAACGAGG", "CTCTGATAAGTAAGTTTCTTTCTCTGCC", 630)
perform_primer_search("cpsJ", "CCTCGACGTATATTCTGGAGAAAC", "GCTTAGTTTCACCAAATGCACGTAG", 553)
perform_primer_search("cpsK", "GCGTTGCACAACGAATTGCTAAATAC", "CGCTACAATATAGTAAGGTAGCTGAATC", 422)

# ======= suspected virulence determinants
perform_primer_search("cylA", "GGTTATGCATCAGATCTCTCAA", "TCTTCAGGTTTAAAATCTGG", 223)
perform_primer_search("cylB", "GGAGAATTAGTGTTTAGAGCG", "GCTTCATAACCATTGTTACTATAGAAAC", 522)
perform_primer_search("cylM", "AAGATTGTCTGTGCCATGGA", "TACTCACTTCCGGCAACCTT", 159)
perform_primer_search("cbh", "CTCATAGGATCCATCACCAACATCAC", "TGGCTGGAATTCACTTTTCAGGCTAT", 580)
perform_primer_search("gelE", "TTGTTGGAAGTTCATGTCTA", "TTCATTGACCAGAACAGATT", 1484)
perform_primer_search("fsrB", "GCATTGTTATCTATGTCGCCATACC", "GGCTTAGTTCCCACACCATC", 396)

# ======= pathogenicity island
perform_primer_search("gls24_like", "GCATTAGATGAGATTGATGGTC", "GCGAGGTTCAGTTTCTTC", 446)
perform_primer_search("esp", "CGATAAAGAAGAGAGCGGAG", "GCAAACTCTACATCCACGTC", 539)
perform_primer_search("psaA", "CTATTTTGCAGCAAGTGATG", "CGCATAGTAACTATCACCATCTTG", 540)

Performing primer-based search for cpsA in Enterococcus faecalis. (FWD=GTAGAAGAAGCAAGCCAGTACGCC, REV=CCTCTGCAGCAATCTGTTTCATGG, len approx. 478)
Will NOT use GFF files; primer PCR hits will be interpreted as gene hits.
100%|███████████████████████████████████| 1081/1081 [02:59<00:00,  6.02genome/s]
Performing primer-based search for cpsB in Enterococcus faecalis. (FWD=GTGTCATCACAGCTATCGTCGC, REV=CCGGCATTGGATAAGGAAATAGCC, len approx. 603)
Will NOT use GFF files; primer PCR hits will be interpreted as gene hits.
100%|███████████████████████████████████| 1081/1081 [03:00<00:00,  6.00genome/s]
Performing primer-based search for cpsC in Enterococcus faecalis. (FWD=CCTGAATATCAATGTATTTGGGCAGTC, REV=CCAACGCTTTGCTTCTTGAATGAC, len approx. 300)
Will NOT use GFF files; primer PCR hits will be interpreted as gene hits.
100%|███████████████████████████████████| 1081/1081 [02:56<00:00,  6.11genome/s]
Performing primer-based search for cpsD in Enterococcus faecalis. (FWD=GGATTCTCTTGTTCAACAAACCATTGG, RE

#### Step 3.3: Convert previous step to marker seed (multi-)FASTA files.

ChronoStrain uses FASTA files to read in marker seeds. The previous step merely creates a catalog of raw gene catalog.

We need to generate a FASTA file, but some cleanup has to be done!

In [51]:
with open(MARKER_SEED_DIR / "manual_seeds.tsv", "wt") as metadata_tsv:
    for feather_file in MARKER_SEED_DIR.glob("*.feather"):
        gene_name = feather_file.stem
        fasta_path = MARKER_SEED_DIR / f'{gene_name}.fasta'
        _df = pd.read_feather(feather_file)
        with open(fasta_path, 'wt') as f:
            for row_idx, row in _df.iterrows():
                record = SeqRecord(
                    Seq(row['GeneSeq']),
                    id="{}_{}".format(gene_name, row_idx),
                    description="Src={}:{}".format(row['Accession'], row['Gene'])
                )
                SeqIO.write([record], f, 'fasta')
        print(f'{feather_file.name} -> {fasta_path.name}')
        
        print(
            "{}\t{}\t{}".format(
                gene_name, fasta_path, f"POLYMORPHIC_{gene_name}"
            ), 
            file=metadata_tsv
        )

cpsK.feather -> cpsK.fasta
cbh.feather -> cbh.fasta
cpsA.feather -> cpsA.fasta
cpsB.feather -> cpsB.fasta
cpsC.feather -> cpsC.fasta
cpsD.feather -> cpsD.fasta
cpsE.feather -> cpsE.fasta
cpsF.feather -> cpsF.fasta
cpsG.feather -> cpsG.fasta
cpsH.feather -> cpsH.fasta
cpsI.feather -> cpsI.fasta
cpsJ.feather -> cpsJ.fasta
cylA.feather -> cylA.fasta
cylB.feather -> cylB.fasta
cylM.feather -> cylM.fasta
esp.feather -> esp.fasta
fsrB.feather -> fsrB.fasta
gelE.feather -> gelE.fasta
gls24_like.feather -> gls24_like.fasta
psaA.feather -> psaA.fasta


#### 3.4 Extract MetaPhlAn markers.

In [77]:
!python python_helpers/extract_metaphlan_markers.py \
    -t "$METAPHLAN_TAXONOMIC_KEY" \
    -i "$METAPHLAN_DB_PATH" \
    -o "$MARKER_SEED_DIR"/metaphlan_seeds.tsv

Searching for marker seeds from MetaPhlAn database: mpa_vJan21_CHOCOPhlAnSGB_202103.
Target # of markers: 400
Found metaphlan marker ID SGB7962__ILGNLFCD_00717.
Found marker `SGB7962__ILGNLFCD_00717` (length 1869)
Found metaphlan marker ID SGB7962__IDAMNEFF_00013.
Found marker `SGB7962__IDAMNEFF_00013` (length 1650)
Found metaphlan marker ID SGB7962__KIHIIFOM_01796.
Found marker `SGB7962__KIHIIFOM_01796` (length 1227)
Found metaphlan marker ID SGB7962__KGLKPILN_02253.
Found marker `SGB7962__KGLKPILN_02253` (length 1224)
Found metaphlan marker ID SGB7962__DCIIIAKG_00503.
Found marker `SGB7962__DCIIIAKG_00503` (length 1209)
Found metaphlan marker ID SGB7962__LMHKBOKG_02316.
Found marker `SGB7962__LMHKBOKG_02316` (length 1209)
Found metaphlan marker ID SGB7962__OIKIMJFJ_01574.
Found marker `SGB7962__OIKIMJFJ_01574` (length 1167)
Found metaphlan marker ID SGB7962__JFHBNJPK_02477.
Found marker `SGB7962__JFHBNJPK_02477` (length 675)
Found metaphlan marker ID SGB7962__CDLKOKGH_00112.
Found ma

#### 3.5 Combine marker seed files.

In [78]:
!cat "$MARKER_SEED_DIR"/mlst_seeds.tsv > $MARKER_SEED_INDEX
!cat "$MARKER_SEED_DIR"/manual_seeds.tsv >> $MARKER_SEED_INDEX
!cat "$MARKER_SEED_DIR"/metaphlan_seeds.tsv >> $MARKER_SEED_INDEX

print("Created Marker seed index: {}".format(MARKER_SEED_INDEX))
assert MARKER_SEED_INDEX.exists()

Created Marker seed index: /mnt/e/infant_nt/database/marker_seeds/marker_seed_index.tsv


### Step 4: Run Chronostrain's make-db command.

By the end of the previous step, we have:

1) FASTA files for each gene, listing out seed sequence(s).
2) A TSV file (marker_seed_index.tsv) containing a list of gene names and the paths to each of these FASTA files.

In [80]:
!env \
    JAX_PLATFORM_NAME=cpu \
    CHRONOSTRAIN_DB_DIR="$CHRONOSTRAIN_DB_DIR" \
    chronostrain -c chronostrain.ini \
      make-db \
      -m $MARKER_SEED_INDEX \
      -r $REFSEQ_INDEX \
      -b $BLAST_DB_NAME -bd $BLAST_DB_DIR \
      --min-pct-idty $MIN_PCT_IDTY \
      --ident-threshold 0.998 \
      -o $CHRONOSTRAIN_TARGET_JSON \
      --threads $NUM_CORES

2023-09-21 15:20:27,939 [INFO - chronostrain.cli] - Creating symbolic links to reference catalog (target dir: /mnt/e/infant_nt/database/chronostrain_files)
2023-09-21 15:20:28,226 [INFO - chronostrain.cli] - Building raw DB using BLAST.
2023-09-21 15:20:28,237 [INFO - chronostrain.cli] - Resolving overlaps.
2023-09-21 15:20:28,623 [INFO - chronostrain.cli] - Created 8 merged markers for strain CP022488.1.
2023-09-21 15:20:28,691 [INFO - chronostrain.cli] - Created 8 merged markers for strain NZ_CP022488.1.
2023-09-21 15:20:28,754 [INFO - chronostrain.cli] - Created 8 merged markers for strain OD940434.1.
2023-09-21 15:20:28,817 [INFO - chronostrain.cli] - Created 8 merged markers for strain OD940440.2.
2023-09-21 15:20:28,877 [INFO - chronostrain.cli] - Created 8 merged markers for strain OD940422.1.
2023-09-21 15:20:28,882 [WARNING - chronostrain.cli] - GFF file not found for LR962855.1.
2023-09-21 15:20:28,883 [WARNING - chronostrain.cli] - GFF file not found for LR962855.1.
2023-09-

# OPTIONAL: pre-load the database files.

ChronoStrain's database is specified by a single JSON file. In this notebook, it is given by the variable `CHRONOSTRAIN_TARGET_JSON`. The `JSONParser` object will attempt to parse the file, and extract the markers (each specified by a start and end position on the assembly) from the chromosomal assembly.

The directory in which the assemblies are stored is in `<data_dir>` (in this notebook it is given by the variable `CHRONOSTRAIN_DB_DIR`), or more specifically `<data_dir>/__<database_name>_/assemblies/<strain_id>/<sequence_acc>.fasta`.
If the data directory does not contain the specified FASTA file, it will attempt to download it from NCBI (by assuming that the sequence accession is an NCBI nucleotide acession).

The loading takes quite a while, and is largely disk I/O-bound. The database will be stored in binary via `pickle`, so that on future usages it will load faster. (try running this cell a second time and see the difference!)

In [ ]:
from chronostrain.database import JSONParser
src_db = JSONParser(
    entries_file=CHRONOSTRAIN_TARGET_JSON,
    data_dir=CHRONOSTRAIN_DB_DIR,
    marker_max_len=50000,
    force_refresh=False
).parse()